In [1]:
import csv
import operator
import json
import math

In [2]:
def read_CSV(path):
    data = []
    with open(path) as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row[1:])
    return data[1:]

In [3]:
def sim(data, user_a, user_b):
    chisl = 0
    znam_a = 0
    znam_b = 0
    for i in range(len(data[0])):
        if data[user_a][i] != -1 and data[user_b][i] != -1:
            chisl += data[user_a][i] * data[user_b][i]
            znam_a += data[user_a][i] ** 2
            znam_b += data[user_b][i] ** 2
    result = chisl / (math.sqrt(znam_a) * math.sqrt(znam_b))
    return result

In [4]:
def most_similars(data, user):
        needed = []
        for v in range(len(data)):
            if user != v:
                needed.append((v, sim(data, user, v)))
        needed = sorted(needed, key=operator.itemgetter(1), reverse=True);
        return needed

In [5]:
def avg_watched(data, user):
    chisl = sum(i for i in data[user] if i != -1)
    znam = sum(1 for i in data[user] if i != -1)
    result = chisl / znam
    return result

In [6]:
def r_ui(data, user, film, k):
    result = 0
    chisl = 0
    znam = 0

    r_u = avg_watched(data, user)
    
    counter = 0
    for v in most_similars(data, user):
        if counter == k: break
        if data[v[0]][film] != -1:
            r_v = avg_watched(data, v[0])
            chisl += v[1] * ((data[v[0]][film]) - r_v)
            znam += abs(v[1])
            counter += 1
    result = chisl / znam + r_u
    return result

In [7]:
def prepare_output(data, context_place, context_day, k):
    best = {}
    grades = {}
    prepared = []
    for user_i in range(len(data)):
        task_1 = {}
        for film_i in range(len(data[user_i])):
            if data[user_i][film_i] == -1:
                task_1["movie " + str(film_i + 1)] = round(r_ui(data, user_i, film_i, k), 3)
                if not (user_i + 1 in grades):
                    grades[user_i + 1] = {}
                grades[user_i + 1]["movie " + str(film_i + 1)] = round(r_ui(data, user_i, film_i, k), 3)
        task_2 = {}
        for user_s in most_similars(data, user_i):
            if user_i + 1 in best: break
            for film_i in range(len(data[user_i])):
                if data[user_i][film_i] == -1 and context_place[user_s[0]][film_i] == " h" and \
                    (context_day[user_s[0]][film_i] == " Sat" or context_day[user_s[0]][film_i] == " Sun") and \
                    data[user_s[0]][film_i] > avg_watched(data, user_s[0]):
                    best[user_i + 1] = "movie: " + str(film_i + 1)
                    task_2["movie " + str(film_i + 1)] = grades[user_i + 1]['movie ' + str(film_i + 1)]
                    break
        tmp = {'user': user_i + 1, 1: task_1, 2: task_2}
        prepared.append(tmp)
    return prepared
    

In [8]:
data = []
for i in read_CSV("data.csv"):    
    data.append(list(map(int, i)))
context_place = read_CSV("context_place.csv")
context_day = read_CSV("context_day.csv")

user_id = int(input("Input user ID: "))
users = prepare_output(data, context_place, context_day, 4)
cur_user = json.dumps(users[user_id - 1])

json_file = open("output.json",'w')
json_file.write(cur_user)
json_file.close()

Input user ID: 1
